# ⚽ Análisis de Estadísticas de Partidos de Fútbol con PySpark

Este notebook aplica técnicas de PySpark (DataFrames, Agregaciones, UDFs y Funciones de Ventana) para el análisis de datos de fútbol.

In [5]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window
from math import radians, cos, sin, asin, sqrt

# --- REEMPLAZO IMPORTANTE: Montar Google Drive ---
# Ejecuta esto si tus archivos están en Drive. Reemplaza 'path_to_data' por la ruta real si usas el método de enlace simbólico.
from google.colab import drive
drive.mount('/content/drive')

# Definir la carpeta base donde se encuentran los CSVs (AJUSTAR SEGÚN TU RUTA DE DRIVE)
# Ejemplo: Si los datos están en 'Mi Drive/Colab Notebooks/data/soccer-data/'
DATA_BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/soccer-data/'

# Creación de la sesión de Spark
spark = (SparkSession.builder
    .appName("Soccer Analytics")
    .getOrCreate())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---

## 🏆 Dataset 1: Resultados de Partidos (`results.csv`)

### Carga y Exploración Inicial

In [6]:
# Definición del esquema para resultados de partidos (para evitar inferencia lenta/errónea)
resultsSchema = StructType([
    StructField("date", DateType(), False),
    StructField("home_team", StringType(), False),
    StructField("away_team", StringType(), False),
    StructField("home_score", IntegerType(), False),
    StructField("away_score", IntegerType(), False),
    StructField("tournament", StringType(), False),
    StructField("city", StringType(), False),
    StructField("country", StringType(), False),
    StructField("neutral", StringType(), False)
])

# Carga del DataFrame de Resultados
results_df = (spark.read.option("header", "true")
              .option("dateFormat", "yyyy-MM-dd")
              .csv(DATA_BASE_PATH + "results.csv", schema=resultsSchema)
             )

results_df.printSchema()
results_df.show(5)
print(f"Número total de partidos: {results_df.count()}")

root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: string (nullable = true)

+----------+--------------+------------------+----------+----------+----------+------------+---------+-------+
|      date|     home_team|         away_team|home_score|away_score|tournament|        city|  country|neutral|
+----------+--------------+------------------+----------+----------+----------+------------+---------+-------+
|2023-10-01|  FC Barcelona|   Athletic Bilbao|         2|         0|  League A|   Barcelona|    Spain|  FALSE|
|2023-10-01|   Real Madrid|   Atletico Madrid|         1|         2|  League A|      Madrid|    Spain|  FALSE|
|2023-10-02|  Boca Juniors|       River Plate|         1|         1|     Cup B|Bu

### Ejercicio 1: Resumen por Torneo

Calcule para cada **Torneo** la fecha del primer partido (`start_date`), la fecha del último partido (`last_date`) y el **número total de goles** marcados (`total_goals`) en dicho torneo.

In [7]:
tournament_summary = (
    results_df
    .withColumn("total_match_goals", col("home_score") + col("away_score"))
    .groupBy("tournament")
    .agg(
        min("date").alias("start_date"),
        max("date").alias("last_date"),
        sum("total_match_goals").alias("total_goals")
    )
)

(tournament_summary
 .orderBy(col("total_goals").desc())
 .show(5, truncate=False)
)

+----------+----------+----------+-----------+
|tournament|start_date|last_date |total_goals|
+----------+----------+----------+-----------+
|League A  |2023-10-01|2023-10-03|12         |
|Cup B     |2023-10-02|2023-10-02|2          |
+----------+----------+----------+-----------+



---

## 🥅 Dataset 2: Goleadores por Partido (`goalscorers.csv`)

### Carga y Exploración Inicial

In [8]:
# Definición del esquema para goleadores
scorersSchema = StructType([
    StructField("date", DateType(), False),
    StructField("home_team", StringType(), False),
    StructField("away_team", StringType(), False),
    StructField("team", StringType(), False),
    StructField("scorer", StringType(), False),
    StructField("minute", IntegerType(), True),
    StructField("own_goal", StringType(), True),
    StructField("penalty", StringType(), True)
])

# Carga del DataFrame de Goleadores
scorers_df = (spark.read.option("header", "true")
              .option("dateFormat", "yyyy-MM-dd")
              .csv(DATA_BASE_PATH + "goalscorers.csv", schema=scorersSchema)
             )

scorers_df.printSchema()
scorers_df.show(5, truncate=False)
print(f"Número total de goles registrados: {scorers_df.count()}")

root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- team: string (nullable = true)
 |-- scorer: string (nullable = true)
 |-- minute: integer (nullable = true)
 |-- own_goal: string (nullable = true)
 |-- penalty: string (nullable = true)

+----------+--------------+---------------+---------------+--------------+------+--------+-------+
|date      |home_team     |away_team      |team           |scorer        |minute|own_goal|penalty|
+----------+--------------+---------------+---------------+--------------+------+--------+-------+
|2023-10-01|FC Barcelona  |Athletic Bilbao|FC Barcelona   |Lewandowski R.|20    |NULL    |NULL   |
|2023-10-01|Real Madrid   |Atletico Madrid|Atletico Madrid|Griezmann A.  |80    |NULL    |NULL   |
|2023-10-02|Boca Juniors  |River Plate    |Boca Juniors   |Cavani E.     |50    |NULL    |NULL   |
|2023-10-02|Manchester Utd|Liverpool      |Manchester Utd |Rashford M.   |45    |NULL    |NU

### Ejercicio 2: Ranking de Goleadores (Agregación)

Calcule el **número total de goles** y la **media de minuto** en el que marca el gol para cada jugador (`scorer`). Muestre el ranking de los 5 mejores goleadores.

In [9]:
scorer_ranking = (
    scorers_df
    .filter(col("own_goal").isNull()) # Excluimos goles en propia puerta
    .groupBy("scorer")
    .agg(
        count("scorer").alias("total_goals"),
        round(avg("minute"), 2).alias("avg_minute_of_goal")
    )
)

(scorer_ranking
 .orderBy(col("total_goals").desc())
 .show(5)
)

+--------------+-----------+------------------+
|        scorer|total_goals|avg_minute_of_goal|
+--------------+-----------+------------------+
|     Cavani E.|          1|              50.0|
|     Mbappe K.|          1|              85.0|
|Lewandowski R.|          1|              20.0|
|  Fernandes B.|          1|              90.0|
|  Griezmann A.|          1|              80.0|
+--------------+-----------+------------------+
only showing top 5 rows



---

### Ejercicio 3: Cálculo de Valor de Gol con UDF

Cree una función de usuario (UDF) llamada `goal_value` que asigne un **valor** a cada gol en función del minuto:

- Gol en el minuto 90 o posterior: **Valor 3** (Gol tardío/decisivo)
- Gol entre el minuto 76 y 89: **Valor 2**
- Gol antes del minuto 76: **Valor 1**

Calcule el valor total de los goles para cada jugador.

In [10]:
# 1. Definición de la función Python
def goal_value_function(minute):
    if minute is None:
        return 0
    elif minute >= 90:
        return 3
    elif minute >= 76:
        return 2
    else:
        return 1

# 2. Conversión a UDF de Spark
goal_value_udf = udf(goal_value_function, IntegerType())

# 3. Aplicación y Agregación
scorer_value = (
    scorers_df
    .filter(col("own_goal").isNull())
    .withColumn("goal_value", goal_value_udf(col("minute")))
    .groupBy("scorer")
    .agg(
        sum("goal_value").alias("total_goal_value"),
        count("scorer").alias("total_goals")
    )
)

(scorer_value
 .orderBy(col("total_goal_value").desc())
 .show(5)
)

+--------------+----------------+-----------+
|        scorer|total_goal_value|total_goals|
+--------------+----------------+-----------+
|  Fernandes B.|               3|          1|
|     Mbappe K.|               2|          1|
|  Griezmann A.|               2|          1|
|     Cavani E.|               1|          1|
|Lewandowski R.|               1|          1|
+--------------+----------------+-----------+
only showing top 5 rows



---

### Ejercicio 4: Goles Acumulados por Equipo (Funciones de Ventana)

Utilizando las **Funciones de Ventana** (`Window`), calcule el **número acumulado de goles** para cada equipo (`team`), ordenados por la fecha (`date`) y el minuto (`minute`) del gol. Esto simula el *running total* de la evolución goleadora.

In [12]:
# Creamos un identificador único para la secuencia de goles (monotonically_increasing_id)
goals_with_id = scorers_df.withColumn("goal_sequence_id", monotonically_increasing_id())

# 1. Definición de la Ventana:
#    - Partición: por equipo ('team').
#    - Orden: por fecha ('date') y luego por el ID de secuencia para un orden determinista.
window_spec = Window.partitionBy('team').orderBy('date', 'goal_sequence_id')

# 2. Cálculo de la Suma Acumulada
acc_goals = (
    goals_with_id
    .filter(col("own_goal").isNull())
    .select(
        'date',
        'team',
        'scorer',
        'minute',
        # Calcula la suma acumulada de goles (contamos 1 por cada fila)
        sum(lit(1)).over(window_spec).alias('cumulative_goals')
    )
)

# 3. Muestra de los resultados (solo para el equipo de Argentina como ejemplo)
(acc_goals
 .filter(col("team") == "Manchester Utd")
 .orderBy("date", "minute")
 .show(10)
)

+----------+--------------+------------+------+----------------+
|      date|          team|      scorer|minute|cumulative_goals|
+----------+--------------+------------+------+----------------+
|2023-10-02|Manchester Utd| Rashford M.|    45|               1|
|2023-10-02|Manchester Utd|Fernandes B.|    90|               2|
+----------+--------------+------------+------+----------------+

